# this is a program to accomplish the following

Select a youtube channel you want to analyze. Then put in its channel id in the cell below. 
If you do not know where to find in the channel id, you can post in a URL from a video from the channel in the link:
https://commentpicker.com/youtube-channel-id.php   to find the channel id.

Then run the code and you will end with a dataframe that all the videos from that and the corresponding stats for those videos: Views, likes, dislikes and comment -counts. You will also get the ratios between views, comments, likes and dislikes.
This is interesting, because those ratios tells us something about the views engagements with the content and channel.


- Put in the channel ID of the creator you want to examine and run the rest of the code.



In [21]:
import requests
import pandas as pd
import numpy as np

YT_API = "Get you own key at youtube api"
from googleapiclient.discovery import build

# run rependencies



In [22]:
youtube = build('youtube', 'v3', developerKey=YT_API)

In [23]:
def get_channel_videos(channel_id):
    res = youtube.channels().list(id=channel_id,
                                 part='contentDetails').execute()
    
    playlist_id = res ['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    
    videos = []
    next_page_token = None
    
    while 1:
        res = youtube.playlistItems().list(playlistId = playlist_id,
                                          part='snippet',
                                          maxResults=50,
                                          pageToken=next_page_token).execute()
        
        videos += res['items']
        next_page_token = res.get('nextPageToken')
        
        if next_page_token is None:
            break
            
    return videos

# here below put in channel ID

In [57]:
# Here you put in the channel ID of the creator you want to examine. 
# The channel id can be found in end of the URL to the channel on youtube

#videos =  get_channel_videos('UCkUq-s6z57uJFUFBvZIVTyg')  # this is with the cid , original of the tutor
# run this one to make the var with all videos used to generate further variables for stats


#videos =  get_channel_videos('UCupvZG-5ko_eiXAupbDfxWw')   # CNN
videos =  get_channel_videos('UCIveFvW-ARp_B_RckhweNJw')   # Steven Crowder

In [58]:
video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))   # this one is needed
# to 
# map all the IDs of the videos in the videos I just found with the other function

In [59]:
def get_videos_stats(video_ids):
    stats = []
    for i in range(0, len(video_ids), 50): # i can pass max 50 videos at a time, therefore the 50
        res = youtube.videos().list(id=','.join(video_ids[i:i+50]), # making list of video IDs
                                    part='statistics').execute()
        stats += res['items']
                     
    return stats 



In [60]:
stats = get_videos_stats(video_ids)

In [61]:
listTitle = []
for video in videos:
    listTitle.append(video['snippet']['title'])
    
titles = pd.DataFrame(listTitle)


In [62]:
titles

,0
0,The Greatest SELF OWN: H3H3 REKTS Himself...BR...
1,Who ACTUALLY Divides Americans?? Don Lemon Pus...
2,Leader of the Free World?? Putin EXPOSES Biden...
3,BIDEN-PUTIN SHOWDOWN: How Will Biden Compare t...
4,Is Jon Stewart a Conspiracy Theorist?! Lab Lea...
...,...
1180,Abortion is SUPER FUN!!
1181,GO GREEN... NOT! Captain Planet & Obama's Poli...
1182,"A Farewell to Bush, LIBERAL CELEBS + Brad Pitt..."
1183,"Hamas children, THE GAYS and Sarah Palin!"


In [63]:
nstats= pd.json_normalize(stats) 
nstats['Title'] = titles

In [64]:
nstats.drop(columns=(['etag','kind','statistics.favoriteCount']),inplace=True)

In [65]:
df = nstats.reindex(columns=['Title', 'id', 'statistics.viewCount', 'statistics.likeCount', 'statistics.dislikeCount', 'statistics.commentCount'])

In [66]:
# float, because when i did to int8 it changed the values, some of them to egatives, so the data was 'useless' as int8. float works
df = df.astype({'statistics.likeCount': np.float, 'statistics.viewCount': np.float, 'statistics.dislikeCount': np.float, 'statistics.commentCount': np.float})

In [67]:
df['Likes per Views'] = df['statistics.likeCount']/df['statistics.viewCount']
df['DisLikes per Views'] = df['statistics.dislikeCount']/df['statistics.viewCount']
df['Comment per Views'] = df['statistics.commentCount']/df['statistics.viewCount']
df['Comment per Like'] = df['statistics.commentCount']/df['statistics.likeCount']
df['Comment per DisLike'] = df['statistics.commentCount']/df['statistics.dislikeCount']
df

,Title,id,statistics.viewCount,statistics.likeCount,statistics.dislikeCount,statistics.commentCount,Likes per Views,DisLikes per Views,Comment per Views,Comment per Like,Comment per DisLike
0,The Greatest SELF OWN: H3H3 REKTS Himself...BR...,WOEV8lIiJ0I,377209.0,49196.0,4743.0,11850.0,0.130421,0.012574,0.031415,0.240873,2.498419
1,Who ACTUALLY Divides Americans?? Don Lemon Pus...,m4KGvco-Yrg,659102.0,44950.0,3498.0,11463.0,0.068199,0.005307,0.017392,0.255017,3.277015
2,Leader of the Free World?? Putin EXPOSES Biden...,ITH54P6b1hg,982008.0,73265.0,1440.0,7404.0,0.074607,0.001466,0.007540,0.101058,5.141667
3,BIDEN-PUTIN SHOWDOWN: How Will Biden Compare t...,djI7BEeb7tk,800996.0,57145.0,960.0,6567.0,0.071342,0.001199,0.008199,0.114918,6.840625
4,Is Jon Stewart a Conspiracy Theorist?! Lab Lea...,vHy6YEB3TxI,981514.0,71912.0,932.0,7905.0,0.073266,0.000950,0.008054,0.109926,8.481760
...,...,...,...,...,...,...,...,...,...,...,...
1180,Abortion is SUPER FUN!!,5atn-FwfNQM,190639.0,5391.0,700.0,3699.0,0.028279,0.003672,0.019403,0.686144,5.284286
1181,GO GREEN... NOT! Captain Planet & Obama's Poli...,aT8Q94LqJcU,71296.0,1841.0,409.0,960.0,0.025822,0.005737,0.013465,0.521456,2.347188
1182,"A Farewell to Bush, LIBERAL CELEBS + Brad Pitt...",pxPMYH9c8gY,59025.0,1666.0,226.0,603.0,0.028225,0.003829,0.010216,0.361945,2.668142
1183,"Hamas children, THE GAYS and Sarah Palin!",b26kD9rmLc4,75934.0,2249.0,755.0,767.0,0.029618,0.009943,0.010101,0.341040,1.015894


In [73]:
df['Comment per Like'].idxmax()

1099

In [70]:
df['Comment per DisLike'].mean()

6.019056991447135

In [72]:
df['Comment per Like'].mean()

0.2019758016888092

In [74]:
df.loc[[1099]]

,Title,id,statistics.viewCount,statistics.likeCount,statistics.dislikeCount,statistics.commentCount,Likes per Views,DisLikes per Views,Comment per Views,Comment per Like,Comment per DisLike
1099,Occupy ASSAULTS in Denver!,ijLYAnvJ2p0,45834.0,559.0,120.0,1645.0,0.012196,0.002618,0.03589,2.942755,13.708333


In [52]:
df.loc[[976]]

,Title,id,statistics.viewCount,statistics.likeCount,statistics.dislikeCount,statistics.commentCount,Likes per Views,DisLikes per Views,Comment per Views,Comment per Like,Comment per DisLike
976,Visa CEO asked about future of cryptocurrency,_G3F6bCixs0,196549.0,1715.0,109.0,3023.0,0.008726,0.000555,0.01538,1.762682,27.733945
